In [1]:
from DataProcessor import DataProcessor
from main import *

Using TensorFlow backend.


In [2]:
language = 'english'
processor = DataProcessor(name='{0}_all_testing'.format(language))
train_path, test_path = datasets_paths(language)
print(train_path)

/Users/uriavron/Dropbox/School/MSc/nlp/git/project/datasets/english/train.conllu


In [3]:
processor.process(train_path)

In [4]:
from POSTaggers import *
tagger = MTLAllFeaturesTagger(processor, n_epochs=5, hidden_size=20)

In [5]:
features = processor.get_features()

word_dict = processor.get_word2idx_dict()
tag_dict = processor.get_tag2idx_dict()
   

feature_dicts = {k: v for k, v in processor.get_features2idx_dicts().items() if k in features}
feature_dicts = feature_dicts.values()

x_train, y_train, y_train_features = processor.preprocess_sample(train_path)

y_train_features = {k: v for k, v in y_train_features.items() if k in features}
y_train_features = y_train_features.values()

x_train = processor.transform_to_one_hot(x_train, len(word_dict))
y_train = processor.transform_to_one_hot(y_train, len(tag_dict))
y_train_features = [processor.transform_to_one_hot(y_train_feature, len(feature_dict)) for
                    y_train_feature, feature_dict in zip(y_train_features, feature_dicts)]

x_test, y_test, y_test_features = processor.preprocess_sample(test_path)

y_test_features = {k: v for k, v in y_test_features.items() if k in features}
y_test_features = y_test_features.values()

x_test = processor.transform_to_one_hot(x_test, len(word_dict))
y_test = processor.transform_to_one_hot(y_test, len(tag_dict))
y_test_features = [processor.transform_to_one_hot(y_test_feature, len(feature_dict)) for
                   y_test_feature, feature_dict in zip(y_test_features, feature_dicts)]

In [6]:
tagger.fit(x_train, [y_train] + y_train_features, callbacks=[])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 40, 4063)     0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 40, 50)       203200      input_1[0][0]                    
__________________________________________________________________________________________________
masking_1 (Masking)             (None, 40, 50)       0           dense_1[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 40, 50)       0           masking_1[0][0]                  
__________________________________________________________________________________________________
dense_2 (D

2914/2914 [==============================] - 6s 2ms/step - loss: 0.9006 - pos_loss: 0.8448 - degree_loss: 0.0085 - number_loss: 0.0054 - prontype_loss: 0.0066 - mood_loss: 0.0027 - tense_loss: 0.0019 - verbform_loss: 0.0080 - case_loss: 0.0016 - person_loss: 0.0023 - definite_loss: 0.0023 - numtype_loss: 0.0032 - gender_loss: 0.0035 - poss_loss: 0.0032 - polarity_loss: 0.0027 - reflex_loss: 0.0037 - pos_acc: 0.7161 - degree_acc: 1.0000 - number_acc: 1.0000 - prontype_acc: 1.0000 - mood_acc: 1.0000 - tense_acc: 1.0000 - verbform_acc: 1.0000 - case_acc: 1.0000 - person_acc: 1.0000 - definite_acc: 1.0000 - numtype_acc: 1.0000 - gender_acc: 1.0000 - poss_acc: 1.0000 - polarity_acc: 1.0000 - reflex_acc: 1.0000


In [7]:
metrics_output = tagger.evaluate_sample(x_test, [y_test] + y_test_features)
print_str = ""
acc = None
for metric, value in metrics_output:
    if "acc" in metric:
        if acc is None:  # only update the first acc
            acc = value
        print_str += "{0}: {1} ".format(metric, value)
        
print(print_str)

769/769 [==============================] - 1s 2ms/step
pos_acc: 0.7623862133534584 degree_acc: 1.0 number_acc: 1.0 prontype_acc: 1.0 mood_acc: 1.0 tense_acc: 1.0 verbform_acc: 1.0 case_acc: 1.0 person_acc: 1.0 definite_acc: 1.0 numtype_acc: 1.0 gender_acc: 1.0 poss_acc: 1.0 polarity_acc: 1.0 reflex_acc: 1.0 


In [8]:
sent = 'The prevalence of discrimination across racial groups in contemporary America:'
tags = tagger.predict_pos(sent)
print(tags)
tags = tagger.predict_pos2(sent)
print(tags)

['DET', 'NOUN', 'ADP', 'NOUN', 'NOUN', 'NOUN', 'NOUN', 'PUNCT', 'NOUN', 'NOUN']
['DET', 'NOUN', 'ADP', 'NOUN', 'NOUN', 'NOUN', 'NOUN', 'PUNCT', 'NOUN', 'NOUN']


In [9]:
# tags = tagger.predict_pos(np.array([onehot_sent]))[0]
# [processor.get_idx2tag_dict()[idx] for idx in tags][:sent_len]